# INSEE's premises

In [1]:
import geopandas
import re
import pandas as pd
import pyproj
from shapely.ops import transform

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib, descartes

from pynsee.sirene import *
from pynsee.geodata import *

In [2]:
# Subscribe to api.insee.fr and get your credentials!
# Save your credentials with init_conn function :
# from pynsee.utils.init_conn import init_conn
# init_conn(insee_key="my_insee_key", insee_secret="my_insee_secret")

# Beware : any change to the keys should be tested after having cleared the cache
# Please do : from pynsee.utils import clear_all_cache; clear_all_cache()"

In [3]:
variable = ["denominationUniteLegale", 'sigleUniteLegale', 'categorieJuridiqueUniteLegale']

insee_pattern = "INSTITUT NATIONAL DE LA STATISTIQUE ET DES ETUDES ECONOMIQUES"
insee_pattern = "INSTITUT NATIONAL DE LA STATISTIQUE"

pattern = [insee_pattern, 'INSEE', '7120']
# 7120 : Service central d'un ministère

data = search_sirene(variable = variable, pattern = pattern, kind="siret")


!!! An error occurred !!!
Query : https://api.insee.fr/entreprises/sirene/V3/siret?q=(denominationUniteLegale:'INSTITUT NATIONAL DE LA STATISTIQUE ET DES ETUDES ECONOMIQUES' AND sigleUniteLegale:'INSEE' AND categorieJuridiqueUniteLegale:'7120')&nombre=1000
{"header":{"statut":400,"message":"Erreur de syntaxe dans le paramètre q=(denominationUniteLegale:'INSTITUT NATIONAL DE LA STATISTIQUE ET DES ETUDES ECONOMIQUES' AND sigleUniteLegale:'INSEE' AND categorieJuridiqueUniteLegale:'7120')"}}

!!! Make sure you have subscribed to all APIs !!!
Click on all APIs' icons one by one, select your application, and click on Subscribe

Error 400



ValueError: {"header":{"statut":400,"message":"Erreur de syntaxe dans le paramètre q=(denominationUniteLegale:'INSTITUT NATIONAL DE LA STATISTIQUE ET DES ETUDES ECONOMIQUES' AND sigleUniteLegale:'INSEE' AND categorieJuridiqueUniteLegale:'7120')"}}

In [ ]:
df = data.get_location()

In [ ]:
def _convert_polygon(geo, crs_in='EPSG:4326', crs_out='EPSG:3857'):
    
    if geo is not None:
        crsIn = pyproj.CRS(crs_in)
        crsOut = pyproj.CRS(crs_out)

        project = pyproj.Transformer.from_crs(crsIn, crsOut, always_xy=True).transform
        geo_converted = transform(project, geo)

        return geo_converted

# convert openstreetmap data from crs 4326 to crs 3857
df['geometry'] = df['geometry'].apply(lambda x: _convert_polygon(x))

df["crs"] = 'EPSG:3857'

# move overseas departement closer to metropolitan France
df = df.translate()

# add a zoom on ile de France region
df = df.zoom()

In [ ]:
gdf = geopandas.GeoDataFrame(df).set_crs('EPSG:3857')
gdf = gdf.reset_index(drop=True)

for i in range(len(gdf.index)):
    gdf.loc[i,'name'] = str(i+1) + ' - ' + gdf.loc[i,'enseigne1Etablissement']
    gdf.loc[i,'i'] = i + 1
    
# cleaning
gdf = gdf[~gdf.geometry.isnull()]

In [ ]:
geodataList =  get_geodata_list()
mapdep = get_geodata('ADMINEXPRESS-COG-CARTO.LATEST:departement')

# move overseas departement closer to metropolitan France
mapdep = mapdep.translate()
# add a zoom on ile de France region
mapdep = mapdep.zoom()

In [ ]:
# conversion to geopandas df
mapdepgeo = geopandas.GeoDataFrame(mapdep).set_crs("EPSG:3857")
mapdepgeo.head()

In [ ]:
# plot
ax = mapdepgeo.plot(color='white', edgecolor='black', figsize = (15,7))

gdf.plot(ax=ax)
plt.title("INSEE's premises in metropolitan France")

for x, y, i, label in zip(gdf.geometry.x, gdf.geometry.y, gdf.i, gdf.name):
    ax.annotate(int(i), xy=(x, y), xytext=(3, 3), textcoords="offset points")

legends = [mpatches.Patch(label=f) for f in gdf.name]
plt.legend(handles=legends, ncol=2, bbox_to_anchor=(1.05, 1))